In [ ]:
# Load packages
import os
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [ ]:
# Load osm_building Features and calculate polygon centroids
home_path = os.getenv('HOME')
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

In [ ]:
# Set the correct CRS and reproject
osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

In [ ]:
# Load the targeted dataset
pois_path = '../dz-datasets/insurance/gam/'
pois = gpd.read_file(pois_path + 'pois_checked_in_out_wilayas.geojson')
pois.head()

In [ ]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

pois = pois.set_crs("EPSG:4326")
pois = pois.to_crs("EPSG:3857")
pois.crs

In [ ]:
# Replace DataFrame geometries with buffers and write it to a file
pois['buffer'] = pois.geometry.buffer(500)
buffers = pois[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
#gpd.GeoDataFrame(buffers).to_file(pois_path + "buffers.geojson", driver='GeoJSON')
buffers.head()

In [ ]:
# Search for buildings that intersect the locations buffers
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')
res_intersection.count()

In [ ]:
res_intersection.head()

In [ ]:
# Write intersected building into a GeoJSON file
res = res_intersection[['num', 'geometry']]
#res.to_file(pois_path + "intersection.geojson", driver='GeoJSON')
res.head()

In [ ]:
# Group results of intersection and add a to_check column (if intersect then doesn't need to check)
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check_osm'] = False
res_grouped.count()

In [ ]:
# Merge with DataFrame and set geometry to centroids
pois_gdf = pois.merge(res_grouped, left_on='num', right_on='num', how='left')
pois_gdf.drop(columns=['buffer'], inplace=True)
pois_gdf.count()

In [ ]:
# Update to_check column :if doesn't intersect then we need to check it manually
pois_gdf['to_check_osm'] = pois_gdf.apply(lambda row: row.to_check_osm != False, axis=1)
pois_gdf[pois_gdf['to_check_osm'] != False]

In [ ]:
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['to_check_osm'] == True)   
final_gdf[final_gdf.to_check == True]

In [ ]:
# Drop unecessary columns
final_gdf = final_gdf.drop(columns=['to_check_osm'], axis=1)

In [ ]:
# Reproject to geograĥic and Write to a GeoPackage file
final_gdf = final_gdf.set_crs("EPSG:3857")
final_gdf = final_gdf.to_crs("EPSG:4326")
final_gdf.to_file(pois_path + "pois_to_check.geojson", driver='GeoJSON')